In [0]:
tiers = ['bronze', 'silver', 'gold']
adls_paths = {tier: f'abfss://{tier}@datapr0ject.dfs.core.windows.net/' for tier in tiers}

bronze_adls = adls_paths['bronze']
silver_adls = adls_paths['silver']
gold_adls = adls_paths['gold']

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(days=1)
end_date = date.today()

In [0]:
season_year = 2025
API_KEY = 123 #hard coded API key for non-premium user

# Dictionary of common sports with their TheSportsDB API league IDs
sports_ids = {
    "fighting": "4443",  # UFC (Ultimate Fighting Championship)
    "soccer": "4328",    # English Premier League (most prominent soccer league)
    "basketball": "4387", # NBA (National Basketball Association)
    "baseball": "4424",   # MLB (Major League Baseball)
    "american_football": "4391", # NFL (National Football League)
    "rugby": "4464",     # Rugby Super League
    "motorsport": "4370", # Formula 1
    "cycling": "4406",    # Tour de France
    "darts": "4415"       # PDC Darts
}

# Function to safely retrieve league ID with fallback
def get_league_id(sport, default_id=None):
    """
    Retrieves league ID from sports_ids dictionary or falls back to API query if not found.
    
    Args:
        sport (str): Sport name (e.g., 'fighting')
        api_key (str): TheSportsDB API key
        default_id (str, optional): Default ID to return if lookup fails
    
    Returns:
        str: League ID or default_id if not found
    """
    try:
        return sports_ids[sport.lower()]
    except KeyError:
        print(f"League ID for {sport} not found in dictionary. Querying API...")
        url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/all_leagues.php"
        response = requests.get(url)
        if response.status_code == 200:
            leagues = response.json().get("leagues", [])
            for league in leagues:
                if league["strSport"].lower() == sport.lower():
                    print(f"Found {sport} league: {league['strLeague']} (ID: {league['idLeague']})")
                    return league["idLeague"]
        print(f"No league found for {sport}. Returning default ID.")
        return default_id

In [0]:
sport_id = get_league_id('Fighting')

In [0]:
url = f"https://www.thesportsdb.com/api/v1/json/123/eventsseason.php?id={sport_id}&s={season_year}"

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json().get('events', [])
    if not data:
        print("No data returned for the specified year or sport.")
    else:
        file_path = f"{bronze_adls}/{start_date}_{sport_id}_data.json"
        
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")